In [1]:
import re, codecs, requests, os, json, nltk, lucene, numpy as np, pandas as pd
#from nltk.sentiment import SentimentAnalyzer
#from nltk.sentiment.util import *
from org.apache.lucene.analysis.standard import StandardTokenizer
from org.apache.lucene.analysis.en import EnglishAnalyzer
from org.apache.lucene.analysis.tokenattributes import CharTermAttribute
from org.apache.lucene.util import Version
from java.io import StringReader
lucene.initVM()

In [2]:
def tokenize(string):
    tokenizer = StandardTokenizer()
    tokenizer.setReader(StringReader(string))
    token = tokenizer.addAttribute(CharTermAttribute.class_)
    tokenizer.reset()
    lucene_tokens = []
    while tokenizer.incrementToken():
        lucene_tokens.append(token.toString())
    return lucene_tokens

def normalize(string):
    analyzer = EnglishAnalyzer()
    tokenstream = analyzer.tokenStream('token', StringReader(string))
    token = tokenstream.addAttribute(CharTermAttribute.class_)
    tokenstream.reset()
    tokens = []
    while tokenstream.incrementToken():
        if not re.search('\d', token.toString()):
            tokens.append(token.toString())
    return tokens

# def get_phonemes(word):
#     try:
#         return arpabet[word][0]
#     except:
#         try:
#             return custom_dict[word]
#         except:
#             print(word)
#             return word
        
def get_speakers_list(speakers_string):
    speakers = speakers_string.upper()
    speakers = re.sub(' AND ', '/', speakers)
    speakers = re.sub(' & ', '/', speakers)
    speakers = re.sub('\[.*\d ', '', speakers)
    speakers = re.sub(' \(.*\)', '', speakers)
    speakers = re.sub('CHORUS ', '', speakers)
    speakers = re.sub(' EXCEPT.*\]', '', speakers)
    speakers = re.sub('ALEXANDER HAMILTON', 'HAMILTON', speakers)
    speakers = re.sub('GEORGE WASHINGTON', 'WASHINGTON', speakers)
    speakers = re.sub('WASHINTON', 'WASHINGTON', speakers)
    speakers = re.sub('AARON BURR', 'BURR', speakers)
    speakers = re.sub('JAMES MADISON', 'MADISON', speakers)
    speakers = re.sub('JAMES', 'MADISON', speakers)    
    speakers = re.sub('THOMAS JEFFERSON', 'JEFFERSON', speakers)
    speakers = re.sub('ELIZA HAMILTON', 'ELIZA', speakers)
    speakers = re.sub('JOHN LAURENS', 'LAURENS', speakers) 
    speakers = re.sub('\[ENS\]', 'ENSEMBLE', speakers)
    speakers = re.sub('\[WASH\]', 'WASHINGTON', speakers)
    speakers = re.sub('\[WASH/', 'WASHINGTON/', speakers)
    speakers = re.sub('/MULL/', '/MULLIGAN/', speakers)
    speakers = re.sub('/LAUR/', '/LAURENS/', speakers)
    speakers = re.sub('/LAF\]', '/LAFAYETTE', speakers)
    speakers = re.sub('\[', '', speakers)
    speakers = re.sub('\]', '', speakers)
    speakers = speakers.strip().split('/')
    return speakers

In [4]:
hamilton_data = '../data/raw/'
hamilton_json = []
for filename in os.listdir(hamilton_data):
    ### TODO: Make encoding word
    with codecs.open(hamilton_data + filename, encoding='utf-8', errors='replace') as f:
        lines = f.readlines()
    track_dict = dict()
    track_dict['act#'], track_dict['track#'], track_dict['track'] = filename[:-4].split('_')
    lyrics = []
    current_speaker = None
    i = 0
    for line in lines:
        if len(line.strip()) == 0: continue
        if re.match('^\[.*\]$', line.strip()):
            current_speaker = get_speakers_list(line)
        else:
            line_dict = {
                'line#': i,
                'speakers': current_speaker,
                'original': line.strip(),
                'tokenized': tokenize(line),
                'normalized': normalize(line)#,
                #'phonemes': get_phonemes(line)
            }
            lyrics.append(line_dict)
            i += 1
    track_dict['lyrics'] = lyrics
    hamilton_json.append(track_dict)

In [5]:
with open('../data/processed/hamilton_data.json', 'w') as f:
    json.dump(hamilton_json, f)